In [1]:
import pandas as pd
import numpy as np

# Loading work from FindingPairs

AgeDB_df = pd.read_csv("createdCSV/AgeDB_pairs.csv", index_col=0)
CASIA_df = pd.read_csv("createdCSV/CASIA_pairs.csv", index_col=0)

In [2]:
AgeDB_df

,ID,Name,Gender,First Image,First Age,First Age Range,Second Image,Second Age,Second Age Range
0,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10044_HelenHunt_32_f.jpg,32,31-40
1,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10045_HelenHunt_33_f.jpg,33,31-40
2,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10046_HelenHunt_34_f.jpg,34,31-40
3,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10057_HelenHunt_44_f.jpg,44,41-50
4,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10047_HelenHunt_34_f.jpg,34,31-40
...,...,...,...,...,...,...,...,...,...
523,566,ElkeSommer,0,9971_ElkeSommer_62_f.jpg,62,61-70,9960_ElkeSommer_42_f.jpg,42,41-50
524,566,ElkeSommer,0,9971_ElkeSommer_62_f.jpg,62,61-70,9946_ElkeSommer_18_f.jpg,18,11-20
525,566,ElkeSommer,0,9973_ElkeSommer_64_f.jpg,64,61-70,9960_ElkeSommer_42_f.jpg,42,41-50
526,566,ElkeSommer,0,9973_ElkeSommer_64_f.jpg,64,61-70,9946_ElkeSommer_18_f.jpg,18,11-20


In [3]:
CASIA_df

,ID,Gender,First Image,First Age,First Age Range,Second Image,Second Age,Second Age Range
0,2,1,000002_00000273.jpg,44.5,41-50,000002_00000274.jpg,46.0,41-50
1,2,1,000002_00000273.jpg,44.5,41-50,000002_00000275.jpg,56.0,51-60
2,2,1,000002_00000273.jpg,44.5,41-50,000002_00000276.jpg,31.5,31-40
3,2,1,000002_00000273.jpg,44.5,41-50,000002_00000277.jpg,34.5,31-40
4,2,1,000002_00000273.jpg,44.5,41-50,000002_00000279.jpg,46.0,41-50
...,...,...,...,...,...,...,...,...
523,10553,1,010553_00490113.jpg,34.0,31-40,010553_00490115.jpg,26.0,21-30
524,10553,1,010553_00490113.jpg,34.0,31-40,010553_00490116.jpg,36.5,31-40
525,10553,1,010553_00490114.jpg,34.5,31-40,010553_00490115.jpg,26.0,21-30
526,10553,1,010553_00490114.jpg,34.5,31-40,010553_00490116.jpg,36.5,31-40


In [91]:
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

def face_similarities(paths):

    first_image = ins_get_image(paths[0])
    second_image = ins_get_image(paths[1])

    first_face = app.get(first_image)
    second_face = app.get(second_image)

    for face in first_face:
        first_feature = face.normed_embedding

    for face in second_face:
        second_feature = face.normed_embedding

    first_feature_np = np.array(first_feature)
    second_feature_np = np.array(second_feature)

    # Cosine similarites
    sims = np.dot(first_feature_np, second_feature_np.T) / (np.linalg.norm(first_feature_np) * np.linalg.norm(second_feature_np))

    return first_feature_np, second_feature_np, sims

In [97]:
from multiprocessing import Pool

first_filename_list_AgeDB = AgeDB_df["First Image"].to_list()
second_filename_list_AgeDB = AgeDB_df["Second Image"].to_list()

AgeDB_filename_pairs = []

for i in range(len(first_filename_list_AgeDB)):
    first_elem = '/home/ivarbl/FaceProject/FaceProject/Images/AgeDB/' + str(first_filename_list_AgeDB[i][:-4])
    second_elem = '/home/ivarbl/FaceProject/FaceProject/Images/AgeDB/' + str(second_filename_list_AgeDB[i][:-4])
    AgeDB_filename_pairs.append((first_elem,second_elem))

app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(256,256))

AgeDB_sims_output = []

with Pool() as pool:
    for output in pool.map(face_similarities, AgeDB_filename_pairs):
        AgeDB_sims_output.append(output)


In [ ]:
First_Image_Features = []
Second_Image_Features = []
Cosine_Similarity = []

for tripple in AgeDB_sims_output:
    First_Image_Features.append(tripple[0])
    Second_Image_Features.append(tripple[1])
    Cosine_Similarity.append(tripple[2])

AgeDB_df["First Image Features"] = First_Image_Features
AgeDB_df["Second Image Features"] = Second_Image_Features
AgeDB_df["Cosine Similarity"] = Cosine_Similarity

In [ ]:
AgeDB_df

In [ ]:
# Saving work
AgeDB_df.to_csv("createdCSV/AgeDB_result.csv")